In [1]:
from transformers import pipeline, AutoTokenizer,AutoModelForSequenceClassification
import shap
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
classifier = pipeline(
    "sentiment-analysis",
    model=model_name,
    device=-1
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

D:\Proyectos\X\ResponsableAI\v2.venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\angel\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

D:\Proyectos\X\ResponsableAI\v2.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [4]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v,padding='max_length',max_length=512,truncation=True) for v in x])
    
    outputs = model(tv)[0].detach().numpy()
    
    scores = (np.exp(outputs).T/ np.exp(outputs).sum(-1)).T
    return scores[:,-1]

In [5]:
explainer = shap.Explainer(f,tokenizer)

In [7]:
def analyze_sentiment(text):
    result = classifier(text)[0]
    return f"Sentiment: {result['label']}, Score: {result['score']}"

In [6]:
def get_shap_values(text):
    return explainer([text])

In [8]:
def visualiza_shap(shap_values):
    plt.figure(figsize=(20,3))
    shap.plots.text(shap_values[0])
    plt.tight_layout()
    plt.show()

In [9]:
def analyze_text(text):
    print("Text: ", text)
    print(analyze_sentiment(text))
    
    shap_value = get_shap_values(text)
    visualiza_shap(shap_value)
    return shap_value

In [17]:
text = "I meet the RUSSIA President"

analyze_text(text)

Text:  I meet the RUSSIA President
Sentiment: POSITIVE, Score: 0.9995201826095581


  0%|          | 0/42 [00:00<?, ?it/s]

<Figure size 2000x300 with 0 Axes>

.values =
array([[-2.32830644e-09, -1.32630719e-03, -6.49694819e-04,
         3.86639033e-04, -1.53996749e-03,  6.62636478e-04,
         2.79396772e-09]])

.base_values =
array([0.04593425])

.data =
(array(['', 'I ', 'meet ', 'the ', 'RUSSIA ', 'President', ''],
      dtype=object),)

In [15]:
text = "i am the lorax i speak for the trees and for some reason they speak vietnamese."

analyze_text(text)

Text:  i am the lorax i speak for the trees and for some reason they speak vietnamese.
Sentiment: POSITIVE, Score: 0.8815779089927673


  0%|          | 0/420 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [01:45, 105.34s/it]              


<Figure size 2000x300 with 0 Axes>

.values =
array([[-2.87150440e-04, -2.87150440e-04, -8.42624751e-04,
        -9.58654040e-04, -1.60730028e-03, -1.58204735e-03,
        -1.24118960e-03, -7.84701842e-04, -9.53104289e-04,
        -4.82448668e-04, -3.81872349e-04, -9.99509008e-04,
         1.13148941e-04, -1.10782951e-03, -1.17802422e-03,
        -1.33384310e-03, -1.22036284e-03, -4.63882810e-04,
        -6.95357914e-04, -1.61923771e-03,  1.86264515e-09]])

.base_values =
array([0.05678938])

.data =
(array(['', 'i ', 'am ', 'the ', 'lo', 'ra', 'x ', 'i ', 'speak ', 'for ',
       'the ', 'trees ', 'and ', 'for ', 'some ', 'reason ', 'they ',
       'speak ', 'vietnamese', '.', ''], dtype=object),)